In [15]:
import json

import requests
from env import SECRET, KEY #secret sauce


In [19]:
# FLICKR API
# flickr.photos.search

URL = 'https://api.flickr.com/services/rest/?method=flickr.photos.search'

# Param
# written as called

# general
per_page = '500'
page = '1'
format_output = 'json' # call exclude _output
extras = "description, license, date_upload, date_taken, owner_name, icon_server, original_format, last_update, geo, tags, url_sq"
nojsoncallback = '1'
has_geo = '1'

# search specific
lat = '47'
lon = '7'
radius = '2'
radius_units = 'km'

tags = ''
min_taken_date = '' # unix
max_taken_date = '' # unix
accuracy = ''


params = dict(api_key= KEY, per_page= per_page, page= page, format= format_output, extras= extras, nojsoncallback= nojsoncallback,
    has_geo= has_geo, accuracy= accuracy, lat= lat, lon= lon, radius= radius
)

print(params)


{'api_key': 'e48b9fd23fc2f1a38619e6d27a9a7359', 'per_page': '500', 'page': '1', 'format': 'json', 'extras': 'description, license, date_upload, date_taken, owner_name, icon_server, original_format, last_update, geo, tags, url_sq', 'nojsoncallback': '1', 'has_geo': '1', 'accuracy': '', 'lat': '47', 'lon': '7', 'radius': '2'}


In [38]:
r = requests.get(url= URL, params= params )
r_json = r.json()

print(f'Status: {r}')
print(json.dumps(r_json, indent=2))

com/4600/38774577054_c5dd1aa184_s.jpg",
        "height_sq": 75,
        "width_sq": 75
      },
      {
        "id": "39452745712",
        "owner": "82667562@N00",
        "secret": "b83d561b9f",
        "server": "4733",
        "farm": 5,
        "title": "DSC01644",
        "ispublic": 1,
        "isfriend": 0,
        "isfamily": 0,
        "license": "0",
        "description": {
          "_content": "                               "
        },
        "dateupload": "1515026269",
        "lastupdate": "1515026315",
        "datetaken": "2017-09-23 11:47:33",
        "datetakengranularity": "0",
        "datetakenunknown": "0",
        "ownername": "Tom Sanchez",
        "iconserver": "29",
        "iconfarm": 1,
        "tags": "schweiz suisse suiza switzerland svizzera 3seenfahrt bielersee murtensee neuenburgersee",
        "originalsecret": "9f2322b4a6",
        "originalformat": "jpg",
        "latitude": "46.991933",
        "longitude": "7.008136",
        "accuracy": "16